In [7]:
# Célula 1: Configurações iniciais e imports
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import cv2
import torch
from ultralytics import YOLO
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import shutil
import yaml

# Configurações globais
BASE_DIR = './'
IMAGE_DIR = os.path.join(BASE_DIR, "antelope")
ANNOTATIONS_FILE = os.path.join(BASE_DIR, "antelope.json")
OUTPUT_DIR = os.path.join(BASE_DIR, "processed")
RESULTS_DIR = os.path.join(BASE_DIR, "results")
TRAIN_DIR = os.path.join(BASE_DIR, "train_data")
YOLO_CONFIG = os.path.join(BASE_DIR, "antelope_pose.yaml")
TRAIN_MODEL_PATH = os.path.join(BASE_DIR, "yolov8n-pose.pt")
TRAINED_MODEL_PATH = os.path.join(BASE_DIR, "yolov8n-pose-antelope.pt")

# Criar diretórios necessários
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(TRAIN_DIR, exist_ok=True)

In [8]:
def create_yolo_config():
    config = f"""
# Caminhos
path: {BASE_DIR}
train: {TRAIN_DIR}
val: {TRAIN_DIR}  # Na prática usar conjunto diferente

# Keypoints
kpt_shape: [8, 3]  # 8 keypoints com (x, y, visibility)

# Classes
names:
  0: antelope

# Parâmetros obrigatórios
nc: 1  # número de classes
flip_idx: [1, 0, 3, 2, 5, 4, 7, 6]  # conexões para flip horizontal
    """
    
    with open(YOLO_CONFIG, 'w') as f:
        f.write(config)
        
    print(f"Configuração YAML criada em: {YOLO_CONFIG}")

create_yolo_config()

Configuração YAML criada em: ./antelope_pose.yaml


In [9]:
# Célula 3: Preparação do dataset no formato YOLO
def prepare_yolo_dataset():
    """Converte anotações para o formato YOLOv8 pose"""
    with open(ANNOTATIONS_FILE) as f:
        annotations = json.load(f)

    print(f"Preparando dataset YOLO em {TRAIN_DIR}...")
    
    for img_name, bboxes in tqdm(annotations.items(), desc="Convertendo anotações"):
        try:
            # Copiar imagem
            src_img = os.path.join(IMAGE_DIR, img_name)
            dst_img = os.path.join(TRAIN_DIR, img_name)
            shutil.copy(src_img, dst_img)
            
            # Criar arquivo de anotações
            txt_path = os.path.join(TRAIN_DIR, os.path.splitext(img_name)[0] + '.txt')
            
            with open(txt_path, 'w') as f_txt:
                for bbox in bboxes:
                    img = Image.open(src_img)
                    img_w, img_h = img.size
                    
                    # Converter bounding box
                    x_center = (bbox['bndbox']['xmin'] + bbox['bndbox']['xmax']) / (2 * img_w)
                    y_center = (bbox['bndbox']['ymin'] + bbox['bndbox']['ymax']) / (2 * img_h)
                    width = (bbox['bndbox']['xmax'] - bbox['bndbox']['xmin']) / img_w
                    height = (bbox['bndbox']['ymax'] - bbox['bndbox']['ymin']) / img_h
                    
                    # Gerar keypoints fictícios (substituir por seus dados reais)
                    keypoints = [
                        0.7, 0.2, 2,  # snout
                        0.6, 0.15, 2,  # head
                        0.5, 0.3, 2,   # neck_base
                        0.3, 0.7, 2,   # front_right_leg
                        0.7, 0.7, 2,   # front_left_leg
                        0.2, 0.8, 2,   # hind_right_leg
                        0.8, 0.8, 2,   # hind_left_leg
                        0.1, 0.4, 2    # tail_base
                    ]
                    
                    line = [0, x_center, y_center, width, height] + keypoints
                    f_txt.write(' '.join(map(str, line)) + '\n')
                    
        except Exception as e:
            print(f"Erro ao processar {img_name}: {e}")
    
    print("Conversão concluída!")

prepare_yolo_dataset()

Preparando dataset YOLO em ./train_data...


Convertendo anotações:   0%|          | 0/152 [00:00<?, ?it/s]

Convertendo anotações: 100%|██████████| 152/152 [00:01<00:00, 90.25it/s]

Conversão concluída!


In [ ]:
def train_model():
    """Executa o treinamento com transfer learning"""
    print("\nIniciando treinamento com transfer learning...")
    
    # Verificar dados
    if not os.path.exists(TRAIN_DIR) or len(os.listdir(TRAIN_DIR)) == 0:
        raise FileNotFoundError("Dados de treino não encontrados!")
    
    # Carregar modelo
    model = YOLO(TRAIN_MODEL_PATH)
    
    # Hiperparâmetros atualizados para YOLOv8.3+
    train_args = {
        'data': YOLO_CONFIG,
        'epochs': 30,
        'imgsz': 256,
        'batch': 8,
        'optimizer': 'Adam',
        'lr0': 0.001,
        'pretrained': True,
        'project': RESULTS_DIR,
        'name': 'antelope_pose',
        'exist_ok': True,
        'device': '0' if torch.cuda.is_available() else 'cpu',
        'freeze': 5,  # Congelar primeiras 5 camadas
        'weight_decay': 0.0005,
        'patience': 4,  # Early stopping
        'label_smoothing': 0.1,
        'fliplr': 0.5,  # Aumento de dados
        'mosaic': 1.0,   # Aumento de dados
    }
    
    try:
        # Treinar modelo
        results = model.train(**train_args)
        
        # Salvar modelo
        model.save(TRAINED_MODEL_PATH)
        
        # Plotar métricas
        results.plot()
        plt.savefig(os.path.join(RESULTS_DIR, 'training_metrics.png'))
        plt.close()
        
        print(f"✅ Treinamento concluído! Modelo salvo em: {TRAINED_MODEL_PATH}")
        
        # Mostrar estatísticas
        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"\nParâmetros treináveis: {trainable_params/total_params:.2%}")
        
        return model
        
    except Exception as e:
        print(f"❌ Erro durante o treinamento: {e}")
        print("Soluções potenciais:")
        print("1. Verifique o formato das anotações nos arquivos .txt")
        print("2. Atualize o Ultralytics: pip install --upgrade ultralytics")
        print("3. Verifique o caminho absoluto das imagens no YAML")
        raise

# Executar treinamento
train_model()


Iniciando treinamento com transfer learning...
New https://pypi.org/project/ultralytics/8.3.99 available  Update with 'pip install -U ultralytics'
WARNING  'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.92  Python-3.11.0 torch-2.6.0+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
engine\trainer: task=pose, mode=train, model=./yolov8n-pose.pt, data=./antelope_pose.yaml, epochs=30, time=None, patience=4, batch=8, imgsz=256, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=./results, name=antelope_pose, exist_ok=True, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=5, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=

train: Scanning C:\Users\Inteli\Downloads\ponderada\train_data.cache... 152 images, 0 backgrounds, 0 corrupt: 100%|██████████| 152/152 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\Inteli\Downloads\ponderada\train_data.cache... 152 images, 0 backgrounds, 0 corrupt: 100%|██████████| 152/152 [00:00<?, ?it/s]


Plotting labels to results\antelope_pose\labels.jpg... 
optimizer: Adam(lr=0.001, momentum=0.937) with parameter groups 63 weight(decay=0.0), 73 weight(decay=0.0005), 72 bias(decay=0.0)
Image sizes 256 train, 256 val
Using 0 dataloader workers
Logging results to results\antelope_pose
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


       7/30         0G      1.307      8.169     0.6764      1.047      1.329         28        256: 100%|██████████| 19/19 [00:11<00:00,  1.64it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:06<00:00,  1.61it/s]

                   all        152        213       0.89      0.812        0.9      0.631       0.15     0.0751     0.0267    0.00367



      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


       8/30         0G      1.386      8.624      0.677      1.058      1.353         31        256:  84%|████████▍ | 16/19 [00:18<00:03,  1.07s/it]

In [ ]:
# Célula 5: Validação do Modelo
def validate_model(model):
    """Avalia o modelo no conjunto de validação"""
    print("\nIniciando validação do modelo...")
    
    metrics = model.val(
        data=YOLO_CONFIG,
        batch=4,
        imgsz=256,
        conf=0.5,
        iou=0.6,
        device='0' if torch.cuda.is_available() else 'cpu'
    )
    
    # Salvar métricas
    with open(os.path.join(RESULTS_DIR, 'validation_metrics.json'), 'w') as f:
        json.dump(metrics.results_dict, f)
    
    print(f"✅ Validação concluída! Precisão mAP50-95: {metrics.box.map:.2f}")
    return metrics

# Executar validação se o modelo existe
if os.path.exists(TRAINED_MODEL_PATH):
    model = YOLO(TRAINED_MODEL_PATH)
    val_metrics = validate_model(model)
else:
    print("Modelo treinado não encontrado. Pulando validação.")


Iniciando validação do modelo...
Ultralytics 8.3.92  Python-3.11.0 torch-2.6.0+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
YOLOv8n-pose summary (fused): 81 layers, 3,120,107 parameters, 0 gradients, 8.5 GFLOPs


val: Scanning C:\Users\Inteli\Downloads\ponderada\train_data.cache... 152 images, 0 backgrounds, 0 corrupt: 100%|██████████| 152/152 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:05<00:00,  7.41it/s]


                   all        152        213      0.969      0.906      0.948      0.813      0.189      0.127       0.11     0.0237
Speed: 0.1ms preprocess, 19.1ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs\pose\val
✅ Validação concluída! Precisão mAP50-95: 0.81


In [ ]:
def train_model():
    """Executa o treinamento com transfer learning"""
    print("\nIniciando treinamento com transfer learning...")
    
    # Verificar dados
    if not os.path.exists(TRAIN_DIR) or len(os.listdir(TRAIN_DIR)) == 0:
        raise FileNotFoundError("Dados de treino não encontrados!")
    
    # Carregar modelo
    model = YOLO(TRAIN_MODEL_PATH)
    
    # Congelar backbone
    for name, param in model.named_parameters():
        if 'backbone' in name:
            param.requires_grad = False
    
    # Hiperparâmetros com early stopping
    train_args = {
        'data': YOLO_CONFIG,
        'epochs': 30,
        'imgsz': 256,
        'batch': 8,
        'optimizer': 'Adam',
        'lr0': 0.001,
        'pretrained': True,
        'project': RESULTS_DIR,
        'name': 'antelope_pose_frozen_backbone',
        'exist_ok': True,
        'device': '0' if torch.cuda.is_available() else 'cpu',
        'freeze': [0, 1, 2, 3, 4],
        'weight_decay': 0.0005,
        'momentum': 0.9,
        'label_smoothing': 0.1,
        'patience': 4,
        'save_period': 1
    }
    
    # Callback corrigido
    class CustomCallback:
        def __init__(self):
            self.best_map = 0.0
            
        def on_train_epoch_end(self, trainer):
            current_map = trainer.validator.metrics.ap50
            print(f"\nÉpoca {trainer.epoch + 1}/{trainer.args.epochs}")
            print(f"mAP50: {current_map:.3f} | Best mAP50: {self.best_map:.3f}")
            
            if current_map > self.best_map:
                self.best_map = current_map
                print("🎯 Novo melhor modelo encontrado!")
    
    # Treinar
    try:
        results = model.train(
            **train_args,
            callbacks={'on_train_epoch_end': CustomCallback().on_train_epoch_end}
        )
    except Exception as e:
        print(f"Erro durante o treinamento: {e}")
        raise
    
    # Salvar modelo
    model.save(TRAINED_MODEL_PATH)
    
    # Plotar métricas corrigido
    if results:
        fig = plt.figure(figsize=(15, 8))
        plt.plot(results.metrics['train/box_loss'], label='Train Loss')
        plt.plot(results.metrics['val/box_loss'], label='Val Loss')
        plt.title('Curva de Aprendizado')
        plt.xlabel('Época')
        plt.ylabel('Perda')
        plt.legend()
        plt.savefig(os.path.join(RESULTS_DIR, 'training_metrics_frozen.png'))
        plt.close()
    
    print(f"Treinamento concluído! Modelo salvo em: {TRAINED_MODEL_PATH}")
    
    # Mostrar parâmetros treináveis
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"\nParâmetros totais: {total_params:,}")
    print(f"Parâmetros treináveis: {trainable_params:,} ({trainable_params/total_params:.2%})")
    
    return model

# Executar treinamento
train_model()


Executando inferência em imagens de ./test_images...

image 1/3 c:\Users\Inteli\Downloads\ponderada\test_images\image.png: 192x256 1 antelope, 76.6ms
image 2/3 c:\Users\Inteli\Downloads\ponderada\test_images\image2.png: 192x256 2 antelopes, 83.8ms
WARNING  Image Read Error c:\Users\Inteli\Downloads\ponderada\test_images\image3.png


ValueError: need at least one array to stack

In [ ]:
# Célula 7: Análise Comparativa
def compare_models():
    """Compara o modelo treinado com o modelo base"""
    print("\nIniciando análise comparativa...")
    
    # Carregar ambos os modelos
    base_model = YOLO(TRAIN_MODEL_PATH)
    trained_model = YOLO(TRAINED_MODEL_PATH)
    
    # Métricas de comparação
    comparison = {
        'base_model': {
            'params': sum(p.numel() for p in base_model.parameters()),
            'size_mb': os.path.getsize(TRAIN_MODEL_PATH) / (1024 * 1024)
        },
        'trained_model': {
            'params': sum(p.numel() for p in trained_model.parameters()),
            'size_mb': os.path.getsize(TRAINED_MODEL_PATH) / (1024 * 1024)
        }
    }
    
    # Plotar comparação
    plt.figure(figsize=(10, 5))
    
    # Tamanho do modelo
    plt.subplot(1, 2, 1)
    plt.bar(['Base', 'Treinado'], 
            [comparison['base_model']['size_mb'], comparison['trained_model']['size_mb']])
    plt.title('Comparação de Tamanho dos Modelos')
    plt.ylabel('Tamanho (MB)')
    
    # Número de parâmetros
    plt.subplot(1, 2, 2)
    plt.bar(['Base', 'Treinado'], 
            [comparison['base_model']['params'], comparison['trained_model']['params']])
    plt.title('Número de Parâmetros')
    plt.ylabel('Parâmetros (milhões)')
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'model_comparison.png'))
    plt.close()
    
    print("✅ Análise comparativa concluída!")
    return comparison

model_comparison = compare_models()


Iniciando análise comparativa...
✅ Análise comparativa concluída!


In [ ]:
# Célula 8: Deploy do Modelo
def prepare_for_deployment(model):
    """Prepara o modelo para produção"""
    print("\nPreparando para deployment...")
    
    # Exportar para formatos diferentes
    model.export(format='onnx', imgsz=256, simplify=True)
    model.export(format='engine', imgsz=256)
    
    # Criar arquivo de configuração
    deploy_config = {
        'model_path': TRAINED_MODEL_PATH,
        'input_size': 256,
        'class_names': ['antelope'],
        'keypoints': [
            'snout', 'head', 'neck_base',
            'front_right_leg', 'front_left_leg',
            'hind_right_leg', 'hind_left_leg',
            'tail_base'
        ]
    }
    
    with open(os.path.join(BASE_DIR, 'deploy_config.json'), 'w') as f:
        json.dump(deploy_config, f, indent=4)
    
    print("✅ Modelo pronto para deployment!")
    print(f"Formatos disponíveis: {os.listdir(os.path.dirname(TRAINED_MODEL_PATH))}")
    
prepare_for_deployment(model)


Preparando para deployment...
Ultralytics 8.3.92  Python-3.11.0 torch-2.6.0+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)

PyTorch: starting from 'yolov8n-pose-antelope.pt' with input shape (1, 3, 256, 256) BCHW and output shape(s) (1, 29, 1344) (6.2 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim', 'onnxruntime'] not found, attempting AutoUpdate...
